<a href="https://colab.research.google.com/github/Jcc329/Jessica_DATA606/blob/main/Clean_Data/Steam_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 2: Data Cleaning

This Notebook aims to clean and prep the data for analysis

### Data cleaning

The final dataset contains several columns with many values (subdictionaries).
To handle these, I will identify all columns containing desired data, remove unneccessary columns and use the apply function to convert the multidimentional columns into their own dataframes that can be appened back onto the original dataframe. 

#### Text Cleaning

Any text data will undergo additional cleaning to prepare it for analysis, including converting the text to lowercase, removing symbols and punctuation, and generally tidying the data.

This concludes the goals of this notebook. A cleaned dataset will be save and the next stage of EDA will occur in the next notebook in this series. 


In [1]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
#import raw data

GameData = pd.read_csv('RawSteamGameData.csv',  engine='python')

In [3]:
GameData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7309 entries, 0 to 7308
Data columns (total 63 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                7309 non-null   int64  
 1   type                      7309 non-null   object 
 2   name                      7301 non-null   object 
 3   steam_appid               7309 non-null   int64  
 4   required_age              7309 non-null   object 
 5   is_free                   7309 non-null   bool   
 6   detailed_description      6664 non-null   object 
 7   about_the_game            6664 non-null   object 
 8   short_description         7177 non-null   object 
 9   supported_languages       7043 non-null   object 
 10  header_image              7309 non-null   object 
 11  website                   4003 non-null   object 
 12  pc_requirements           7309 non-null   object 
 13  mac_requirements          7309 non-null   object 
 14  linux_re

In [4]:
GameData.isnull().sum()

Unnamed: 0                     0
type                           0
name                           8
steam_appid                    0
required_age                   0
is_free                        0
detailed_description         645
about_the_game               645
short_description            132
supported_languages          266
header_image                   0
website                     3306
pc_requirements                0
mac_requirements               0
linux_requirements             0
developers                   677
publishers                     0
price_overview              2026
packages                    1943
package_groups                 0
platforms                      0
categories                   306
genres                       802
screenshots                  657
movies                      2971
release_date                   0
support_info                   0
background                   657
content_descriptors            0
Review Score                  18
Review Sco

In [5]:
#drop fields with a lot of mising data and fields that are unneccessary for the current analysis.
# Dropping 'drm_notice', 'alternate_appid','score_rank', 'ext_user_account_notice', 'demos', 'dlc', 'recommendations  high number of missing data
# Dropping 'legal_notice', 'header_image', 'website', 'pc_requirements', 'packages','mac_requirements', 'linux_requirements', 'screenshots', 'movies', due to irrelevance
#Dropping duplicated developer, publisher, reviews, price overview, languages
#Acheivements, dlc, and controller support are all covered in categories
#Only 250 games include metacritic scores. I will keep the field however, as I may decide to use it during exploratory analysis, or if I expand the number of games collected

GameData = GameData[['Unnamed: 0', 'type', 'name', 'steam_appid', 'required_age', 'is_free',
       'detailed_description', 'about_the_game', 'short_description',
       'supported_languages', 'developers', 'publishers', 'platforms',
       'categories', 'genres', 'release_date','content_descriptors', 'Review Score',
       'Review Score Description', 'Top Reviews by Upvotes', 'appid',
       'positive', 'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
       'ccu', 'genre', 'tags', 'metacritic'
       ]]

In [6]:
GameData.describe(include = 'all')

,Unnamed: 0,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,developers,publishers,platforms,categories,genres,release_date,content_descriptors,Review Score,Review Score Description,Top Reviews by Upvotes,appid,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,genre,tags,metacritic
count,7.309000e+03,7309,7301,7.309000e+03,7309,7309,6664,6664,7177,7043,6632,7309,7309,7003,6507,7309,7309,7291.000000,7291,3743,7.291000e+03,7291.000000,7291.000000,7291.000000,7291,7291.000000,7291.000000,7291.000000,7291.000000,6779.000000,6779.000000,6779.000000,7291.000000,6018,7291,250
unique,NaN,10,6711,NaN,12,2,6083,6083,6525,1429,4472,3731,5,1314,685,2501,620,NaN,19,3441,NaN,NaN,NaN,NaN,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,621,3336,235
top,NaN,game,System Voice - BAIKEN,NaN,0,False,In this DLC you find full soundtrack from the ...,In this DLC you find full soundtrack from the ...,In this DLC you find full soundtrack from the ...,English<strong>*</strong><br><strong>*</strong...,['TigerQiuQiu'],[''],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}]","{'coming_soon': False, 'date': ''}","{'ids': [], 'notes': None}",NaN,No user reviews,6/10,NaN,NaN,NaN,NaN,"0 .. 20,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Action,[],"{'score': 79, 'url': 'https://www.metacritic.c..."
freq,NaN,4089,5,NaN,7038,6676,9,9,9,1949,151,1311,5234,1138,353,94,6347,NaN,3548,11,NaN,NaN,NaN,NaN,6382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,351,3150,3
mean,9.581954e+05,NaN,NaN,9.995097e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.545878,NaN,NaN,9.997428e+05,813.404197,109.467151,0.169387,NaN,64.316692,3.386778,57.441092,3.637361,632.102522,657.522791,3.086296,44.250171,NaN,NaN,NaN
std,5.116008e+05,NaN,NaN,4.969698e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.829286,NaN,NaN,4.970207e+05,14050.711455,2164.124272,3.767921,NaN,1085.400584,55.440596,1050.501621,59.419759,1093.036894,1104.797742,12.862200,1134.643738,NaN,NaN,NaN
min,7.000000e+01,NaN,NaN,7.000000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,7.000000e+01,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
25%,5.491800e+05,NaN,NaN,5.957540e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,5.959720e+05,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,99.000000,99.000000,0.000000,0.000000,NaN,NaN,NaN
50%,9.197300e+05,NaN,NaN,9.629600e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,9.629900e+05,1.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,299.000000,299.000000,0.000000,0.000000,NaN,NaN,NaN
75%,1.394830e+06,NaN,NaN,1.427740e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,1.428240e+06,20.000000,6.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,799.000000,999.000000,0.000000,0.000000,NaN,NaN,NaN


In [7]:
GameData.head()

,Unnamed: 0,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,developers,publishers,platforms,categories,genres,release_date,content_descriptors,Review Score,Review Score Description,Top Reviews by Upvotes,appid,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,genre,tags,metacritic
0,1212600,game,Budo War Girl: maid of desire,1212600,0,False,"Young madam, AI beast girl, Chinese girl, Insa...","Young madam, AI beast girl, Chinese girl, Insa...",you will get 500 diamonds to start the shop.Ma...,"English, Japanese<strong>*</strong>, Simplifie...",['武道戰姬製作委員會'],['武道戰姬製作委員會'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': 'Jan 23, 2020'}","{'ids': [1, 3, 5], 'notes': 'The content of th...",0.0,5 user reviews,[h1]At a Glance[/h1]\n[table]\n [tr]\n ...,1212600.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,699.0,699.0,0.0,0.0,"Action, Casual, Indie, RPG, Simulation",[],NaN
1,1453070,dlc,Sinister Halloween - Asylum DLC,1453070,0,False,During the events on Halloween night in Sinist...,During the events on Halloween night in Sinist...,"As a journalist, you receive a secret email fr...",English<strong>*</strong><br><strong>*</strong...,['Celeritas Games'],['Celeritas Games'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': 'Oct 30, 2020'}","{'ids': [2, 5], 'notes': 'Dead corpses\r\nBloo...",0.0,6 user reviews,So I wasn't sure if I should give this a posit...,1453070.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0,"Action, Adventure, Indie",[],NaN
2,384441,dlc,Sinister Halloween - Asylum DLC,1453070,0,False,During the events on Halloween night in Sinist...,During the events on Halloween night in Sinist...,"As a journalist, you receive a secret email fr...",English<strong>*</strong><br><strong>*</strong...,['Celeritas Games'],['Celeritas Games'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': 'Oct 30, 2020'}","{'ids': [2, 5], 'notes': 'Dead corpses\r\nBloo...",0.0,6 user reviews,So I wasn't sure if I should give this a posit...,1453070.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0,"Action, Adventure, Indie",[],NaN
3,1644840,game,Kittens and Yarn,1644840,0,False,<h1>🐾 Also Recommended for You: 🐾</h1><p><a hr...,Lots of cuteness and mess! That's exactly what...,Move the pieces of place and untangle the wool...,"English, Portuguese - Brazil, Simplified Chinese",['Pinel Games'],['Pinel Games'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}]","{'coming_soon': False, 'date': 'Jun 17, 2021'}","{'ids': [], 'notes': None}",8.0,Very Positive,A sweet little time waster when you just want ...,1644840.0,94.0,5.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,199.0,199.0,0.0,0.0,Casual,"{'Cats': 113, 'Cute': 107, 'Minimalist': 102, ...",NaN
4,429040,game,Furfly,429040,0,False,"Furfly is a fascinating, black and white world...","Furfly is a fascinating, black and white world...",Help the little furry friend to escape the evi...,English<strong>*</strong><br><strong>*</strong...,['SMD Gaming Studio'],['SMD Gaming Studio'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': 'Dec 18, 2015'}","{'ids': [], 'notes': None}",0.0,9 user reviews,This is actually a very tough game to play. Th...,429040.0,16.0,4.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,499.0,499.0,0.0,0.0,"Action, Adventu

## A major issue with this data are the collumns that contain multiple datapoints. There are 3 formats this takes in  the data frame right now:
- a dictionary
- a list stored as a string
- a list of dictionaries stored as strings

I also need to convert the acheivements value into a binary (I don't care what the acheivements are, only whether or not it's a feature in the game). 

In [8]:
#Expand columns containing multiple datapoints
#https://stackoverflow.com/questions/38231591/split-explode-a-column-of-dictionaries-into-separate-columns-with-pandas

# I will start by handling dictionary lists stored as strings 
# create a function to convert the lists of dictionaries stored as strings to dictionaries for processing
# https://www.geeksforgeeks.org/python-convert-string-to-list-of-dictionaries/

def makelist(DictStrList):
    ''' 
    Takes list of dictionaries stored as a string and converts to a list of dictionaries
    '''
    try:    
        if len(DictStrList.split(', '))==2:
            x = eval(DictStrList)
        else:
            x = list(eval(DictStrList))
    except: 
        x = DictStrList
        # print(DictStrList)
    return x

In [9]:
# Handle categories
#desired output 1 column for each category type with a value indicating whether or not the game includes that value
#Start by finding each category name:
GameData2 = GameData
GameData2['categories'] = GameData2['categories'].apply(makelist)
GameData2 = GameData2.explode('categories')
categories = pd.concat([GameData2.drop(['categories'], axis=1), GameData2['categories'].apply(pd.Series)], axis=1)
uniquecats = list(categories['description'].unique()) #create list of unique categories
uniquecats #remove categories not in english

['Single-player',
 'Downloadable Content',
 'Steam Achievements',
 'Partial Controller Support',
 nan,
 'Multi-player',
 'PvP',
 'Online PvP',
 'Shared/Split Screen PvP',
 'Shared/Split Screen',
 'Cross-Platform Multiplayer',
 'Remote Play Together',
 'Co-op',
 'Online Co-op',
 'Shared/Split Screen Co-op',
 'Full controller support',
 'Steam Trading Cards',
 'Steam Workshop',
 'Steam Cloud',
 'Stats',
 'Steam Leaderboards',
 'Includes level editor',
 'Remote Play on TV',
 'MMO',
 'In-App Purchases',
 'VR Support',
 'Native Steam Controller Support',
 'Captions available',
 'Additional High-Quality Audio',
 'LAN Co-op',
 'Remote Play on Tablet',
 'Game demo',
 'LAN PvP',
 'Remote Play on Phone',
 '单人',
 'Steam 成就',
 '完全支持控制器',
 'Steam 集换式卡牌',
 'Steam 云',
 'Steam Turn Notifications',
 'Commentary available',
 'Valve Anti-Cheat enabled',
 'Mods',
 '部分支持控制器',
 'DLC',
 'SteamVR Collectibles',
 'Includes Source SDK',
 '多人',
 '合作',
 '包含关卡编辑器',
 'Steam 排行榜',
 '玩家对战',
 '线上玩家对战',
 '同屏/分屏玩家对战',
 

In [10]:
#remove categories not in english
remove = ['多人', '合作','部分支持控制器', '包含关卡编辑器', 'Steam 排行榜', '玩家对战', '线上玩家对战', '同屏/分屏玩家对战', '同屏/分屏',
       '统计数据', '远程同乐', '在线合作', 'Steam 创意工坊', '单人', 'Steam 成就', '完全支持控制器', 'Steam 集换式卡牌','Steam 云']
catlist = [cat for cat in uniquecats if cat not in remove]
#remove nan value
catlist = [cat for cat in catlist if pd.isnull(cat) == False]
catlist

['Single-player',
 'Downloadable Content',
 'Steam Achievements',
 'Partial Controller Support',
 'Multi-player',
 'PvP',
 'Online PvP',
 'Shared/Split Screen PvP',
 'Shared/Split Screen',
 'Cross-Platform Multiplayer',
 'Remote Play Together',
 'Co-op',
 'Online Co-op',
 'Shared/Split Screen Co-op',
 'Full controller support',
 'Steam Trading Cards',
 'Steam Workshop',
 'Steam Cloud',
 'Stats',
 'Steam Leaderboards',
 'Includes level editor',
 'Remote Play on TV',
 'MMO',
 'In-App Purchases',
 'VR Support',
 'Native Steam Controller Support',
 'Captions available',
 'Additional High-Quality Audio',
 'LAN Co-op',
 'Remote Play on Tablet',
 'Game demo',
 'LAN PvP',
 'Remote Play on Phone',
 'Steam Turn Notifications',
 'Commentary available',
 'Valve Anti-Cheat enabled',
 'Mods',
 'DLC',
 'SteamVR Collectibles',
 'Includes Source SDK']

In [11]:
# GameData['categories']

In [12]:
#Next iterate through list and create lists for each category to append
def valueoffered(dfCol, valuelist):
    ''' 
    Takes a data frame column in string  format and list of strings and iterates through the column looking for those strings
    returns a dataframe with a column for each value.
    '''
    valueDict = {}
    for value in valuelist: #start a list in a dictionary for each category or other variable
        valueDict[value] = []
    print(valueDict)
    for row in dfCol:
        try:
            for value in valuelist:
                if value in row: #if the game has that category tag, add a 1 to the category column, else add a 0
                    valueDict[value].append(1)
                else:
                    valueDict[value].append(0)
        except: #handle Nans
            for value in valuelist:
                valueDict[value].append(0)
    return pd.DataFrame(valueDict)

CategoryDF = valueoffered(GameData['categories'].astype(str), catlist)
CategoryDF

{'Single-player': [], 'Downloadable Content': [], 'Steam Achievements': [], 'Partial Controller Support': [], 'Multi-player': [], 'PvP': [], 'Online PvP': [], 'Shared/Split Screen PvP': [], 'Shared/Split Screen': [], 'Cross-Platform Multiplayer': [], 'Remote Play Together': [], 'Co-op': [], 'Online Co-op': [], 'Shared/Split Screen Co-op': [], 'Full controller support': [], 'Steam Trading Cards': [], 'Steam Workshop': [], 'Steam Cloud': [], 'Stats': [], 'Steam Leaderboards': [], 'Includes level editor': [], 'Remote Play on TV': [], 'MMO': [], 'In-App Purchases': [], 'VR Support': [], 'Native Steam Controller Support': [], 'Captions available': [], 'Additional High-Quality Audio': [], 'LAN Co-op': [], 'Remote Play on Tablet': [], 'Game demo': [], 'LAN PvP': [], 'Remote Play on Phone': [], 'Steam Turn Notifications': [], 'Commentary available': [], 'Valve Anti-Cheat enabled': [], 'Mods': [], 'DLC': [], 'SteamVR Collectibles': [], 'Includes Source SDK': []}


,Single-player,Downloadable Content,Steam Achievements,Partial Controller Support,Multi-player,PvP,Online PvP,Shared/Split Screen PvP,Shared/Split Screen,Cross-Platform Multiplayer,Remote Play Together,Co-op,Online Co-op,Shared/Split Screen Co-op,Full controller support,Steam Trading Cards,Steam Workshop,Steam Cloud,Stats,Steam Leaderboards,Includes level editor,Remote Play on TV,MMO,In-App Purchases,VR Support,Native Steam Controller Support,Captions available,Additional High-Quality Audio,LAN Co-op,Remote Play on Tablet,Game demo,LAN PvP,Remote Play on Phone,Steam Turn Notifications,Commentary available,Valve Anti-Cheat enabled,Mods,DLC,SteamVR Collectibles,Includes Source SDK
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7304,1,0,1,0,1,1,0,1,1,0,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7305,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7306,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7307,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
#add category df to GameData
GameData_Categories = pd.concat([GameData, CategoryDF], axis=1)

In [14]:
GameData_Categories

,Unnamed: 0,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,developers,publishers,platforms,categories,genres,release_date,content_descriptors,Review Score,Review Score Description,Top Reviews by Upvotes,appid,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,genre,tags,metacritic,Single-player,Downloadable Content,Steam Achievements,Partial Controller Support,Multi-player,PvP,Online PvP,Shared/Split Screen PvP,Shared/Split Screen,Cross-Platform Multiplayer,Remote Play Together,Co-op,Online Co-op,Shared/Split Screen Co-op,Full controller support,Steam Trading Cards,Steam Workshop,Steam Cloud,Stats,Steam Leaderboards,Includes level editor,Remote Play on TV,MMO,In-App Purchases,VR Support,Native Steam Controller Support,Captions available,Additional High-Quality Audio,LAN Co-op,Remote Play on Tablet,Game demo,LAN PvP,Remote Play on Phone,Steam Turn Notifications,Commentary available,Valve Anti-Cheat enabled,Mods,DLC,SteamVR Collectibles,Includes Source SDK
0,1212600,game,Budo War Girl: maid of desire,1212600,0,False,"Young madam, AI beast girl, Chinese girl, Insa...","Young madam, AI beast girl, Chinese girl, Insa...",you will get 500 diamonds to start the shop.Ma...,"English, Japanese<strong>*</strong>, Simplifie...",['武道戰姬製作委員會'],['武道戰姬製作委員會'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': 'Jan 23, 2020'}","{'ids': [1, 3, 5], 'notes': 'The content of th...",0.0,5 user reviews,[h1]At a Glance[/h1]\n[table]\n [tr]\n ...,1212600.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,699.0,699.0,0.0,0.0,"Action, Casual, Indie, RPG, Simulation",[],NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1453070,dlc,Sinister Halloween - Asylum DLC,1453070,0,False,During the events on Halloween night in Sinist...,During the events on Halloween night in Sinist...,"As a journalist, you receive a secret email fr...",English<strong>*</strong><br><strong>*</strong...,['Celeritas Games'],['Celeritas Games'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': 'Oct 30, 2020'}","{'ids': [2, 5], 'notes': 'Dead corpses\r\nBloo...",0.0,6 user reviews,So I wasn't sure if I should give this a posit...,1453070.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0,"Action, Adventure, Indie",[],NaN,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,384441,dlc,Sinister Halloween - Asylum DLC,1453070,0,False,During the events on Halloween night in Sinist...,During the events on Halloween night in Sinist...,"As a journalist, you receive a secret email fr...",English<strong>*</strong><br><strong>*</strong...,['Celeritas Games'],['Celeritas Games'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': 'Oct 30, 2020'}","{'ids': [2, 5], 'notes': 'Dead corpses\r\nBloo...",0.0,6 user reviews,So I wasn't sure if I should give this a posit...,1453070.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0,"Action, Adventure, Indie",[],NaN,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1644840,game,Kittens and Yarn,1644840,0,False,<h1>🐾 Also Recommended for You: 🐾</h1><p><a hr...,Lots of cuteness and mess! That's exactly what...,Move the pieces of place and untangle the wool...,"English, Portuguese - Brazil, Simplified Chinese",['Pinel Games'],['Pinel Games'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}]","{'coming_soon': False, 'dat

In [15]:
#handle genres
GameData2 = GameData
GameData2['genres'] = GameData2['genres'].apply(makelist)
GameData2 = GameData2.explode('genres')
genres =  GameData2['genres'].apply(pd.Series)
uniquegenres = list(genres['description'].unique()) #create list of unique genres
uniquegenres #remove genres not in english


['Action',
 'Casual',
 'Indie',
 'RPG',
 'Simulation',
 'Adventure',
 nan,
 'Strategy',
 'Early Access',
 'Free to Play',
 'Massively Multiplayer',
 'Racing',
 'Sports',
 'Movie',
 'Utilities',
 'Design & Illustration',
 'Web Publishing',
 'Game Development',
 'Sexual Content',
 'Nudity',
 'Violent',
 'Education',
 '冒险',
 'Software Training',
 'Video Production',
 'Gore',
 'Episodic',
 '休闲',
 '动作',
 '独立',
 'Animation & Modeling',
 'Audio Production',
 'Photo Editing',
 'Documentary',
 '模拟',
 '免费开玩',
 'Tutorial',
 'Accounting',
 '抢先体验',
 '策略',
 '角色扮演',
 'Short']

In [16]:
#handle genres
#remove genres not in english
remove = ['抢先体验','策略','角色扮演','模拟', '免费开玩','休闲', '动作', '独立','冒险',]
genlist = [gen for gen in uniquegenres if gen not in remove]
#remove nan value
genlist = [gen for gen in genlist if pd.isnull(gen) == False]
genlist

['Action',
 'Casual',
 'Indie',
 'RPG',
 'Simulation',
 'Adventure',
 'Strategy',
 'Early Access',
 'Free to Play',
 'Massively Multiplayer',
 'Racing',
 'Sports',
 'Movie',
 'Utilities',
 'Design & Illustration',
 'Web Publishing',
 'Game Development',
 'Sexual Content',
 'Nudity',
 'Violent',
 'Education',
 'Software Training',
 'Video Production',
 'Gore',
 'Episodic',
 'Animation & Modeling',
 'Audio Production',
 'Photo Editing',
 'Documentary',
 'Tutorial',
 'Accounting',
 'Short']

In [17]:
GenreDF = valueoffered(GameData['genres'].astype(str),genlist)
GenreDF.head()

{'Action': [], 'Casual': [], 'Indie': [], 'RPG': [], 'Simulation': [], 'Adventure': [], 'Strategy': [], 'Early Access': [], 'Free to Play': [], 'Massively Multiplayer': [], 'Racing': [], 'Sports': [], 'Movie': [], 'Utilities': [], 'Design & Illustration': [], 'Web Publishing': [], 'Game Development': [], 'Sexual Content': [], 'Nudity': [], 'Violent': [], 'Education': [], 'Software Training': [], 'Video Production': [], 'Gore': [], 'Episodic': [], 'Animation & Modeling': [], 'Audio Production': [], 'Photo Editing': [], 'Documentary': [], 'Tutorial': [], 'Accounting': [], 'Short': []}


,Action,Casual,Indie,RPG,Simulation,Adventure,Strategy,Early Access,Free to Play,Massively Multiplayer,Racing,Sports,Movie,Utilities,Design & Illustration,Web Publishing,Game Development,Sexual Content,Nudity,Violent,Education,Software Training,Video Production,Gore,Episodic,Animation & Modeling,Audio Production,Photo Editing,Documentary,Tutorial,Accounting,Short
0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
#add Genre df to GameData
GameData = pd.concat([GameData, GenreDF], axis = 1)

## Next I will handle fields that are stored as dictionaries by turning the column of dictionaries into a dataframe and appending to the original dataset.

In [19]:
# convert nans to empty dict
def handleNaN(col):
    newlist = []
    for item in col:
        if str(item) == 'nan':
            newlist.append({})
        else:
            newlist.append(item)
    return newlist

GameData['metacritic'] = handleNaN(GameData['metacritic'])
GameData['platforms'] = handleNaN(GameData['platforms'])
GameData['release_date'] = handleNaN(GameData['release_date'])
GameData['content_descriptors'] = handleNaN(GameData['content_descriptors'])


In [20]:
# https://stackoverflow.com/questions/38231591/split-explode-a-column-of-dictionaries-into-separate-columns-with-pandas

platforms = pd.json_normalize(GameData['platforms'].astype(str).apply(eval))
release = pd.json_normalize(GameData['release_date'].astype(str).apply(eval))
content = pd.json_normalize(GameData['content_descriptors'].astype(str).apply(eval))
metacritic = pd.json_normalize(GameData['metacritic'].astype(str).apply(eval))
#Rename columns
release = release.rename({'date':'Release Date'})
content = content.rename({'notes': 'Content Notes'})
content = content.drop(columns = 'ids')
metacritic = metacritic.rename({'score':'Metacritic Score'})
metacritic = metacritic.drop(columns = 'url')

In [21]:
#Add all new data frames back into the original

GameData = pd.concat([GameData, platforms, release, content, metacritic], axis = 1)

In [22]:
GameData.head()

,Unnamed: 0,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,developers,publishers,platforms,categories,genres,release_date,content_descriptors,Review Score,Review Score Description,Top Reviews by Upvotes,appid,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,genre,tags,metacritic,Action,Casual,Indie,RPG,Simulation,Adventure,Strategy,Early Access,Free to Play,Massively Multiplayer,Racing,Sports,Movie,Utilities,Design & Illustration,Web Publishing,Game Development,Sexual Content,Nudity,Violent,Education,Software Training,Video Production,Gore,Episodic,Animation & Modeling,Audio Production,Photo Editing,Documentary,Tutorial,Accounting,Short,windows,mac,linux,coming_soon,date,notes,score
0,1212600,game,Budo War Girl: maid of desire,1212600,0,False,"Young madam, AI beast girl, Chinese girl, Insa...","Young madam, AI beast girl, Chinese girl, Insa...",you will get 500 diamonds to start the shop.Ma...,"English, Japanese<strong>*</strong>, Simplifie...",['武道戰姬製作委員會'],['武道戰姬製作委員會'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': 'Jan 23, 2020'}","{'ids': [1, 3, 5], 'notes': 'The content of th...",0.0,5 user reviews,[h1]At a Glance[/h1]\n[table]\n [tr]\n ...,1212600.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,699.0,699.0,0.0,0.0,"Action, Casual, Indie, RPG, Simulation",[],{},1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,"Jan 23, 2020",The content of the game is not suitable for pu...,NaN
1,1453070,dlc,Sinister Halloween - Asylum DLC,1453070,0,False,During the events on Halloween night in Sinist...,During the events on Halloween night in Sinist...,"As a journalist, you receive a secret email fr...",English<strong>*</strong><br><strong>*</strong...,['Celeritas Games'],['Celeritas Games'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': 'Oct 30, 2020'}","{'ids': [2, 5], 'notes': 'Dead corpses\r\nBloo...",0.0,6 user reviews,So I wasn't sure if I should give this a posit...,1453070.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0,"Action, Adventure, Indie",[],{},1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,"Oct 30, 2020",Dead corpses\r\nBlood\r\nGore\r\nBody parts\r\n,NaN
2,384441,dlc,Sinister Halloween - Asylum DLC,1453070,0,False,During the events on Halloween night in Sinist...,During the events on Halloween night in Sinist...,"As a journalist, you receive a secret email fr...",English<strong>*</strong><br><strong>*</strong...,['Celeritas Games'],['Celeritas Games'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': 'Oct 30, 2020'}","{'ids': [2, 5], 'notes': 'Dead corpses\r\nBloo...",0.0,6 user reviews,So I wasn't sure if I should give this a posit...,1453070.0,0.0,0.0,0.0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0,"Action, Adventure, Indie",[],{},1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,False,False,False,"Oct 30, 2020",Dead corpses\r\nBlood\r\nGore\r\nBody parts\r\n,NaN
3,1644840,game,Kittens and Yarn,1644840,0,False,<h1>🐾 Also Recommended for You: 🐾</h1><p><a hr...,Lots of cuteness and mess! That's exactly what...,Move the pieces of place and untangle the wool...,"English, Portuguese - Brazil, Simplified Chinese",['Pinel Games'],['Pinel Games'],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}]","{'coming_soon': False, 'date': 'Jun 17, 2021'}","{'ids': [], 'notes': None}",8.0,

#Text Data Cleaning

Target fields:
- detailed description
- short description
- about the game
- top reviews by upvote
- tags (note, I am treating tags as a text field because there are so many possible tags a game can have)

Initial Cleaning goals:
- Send to lowercase
- remove punctuation and symbols
- remove stopwords

In [23]:
import string
import re
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
# https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string
# https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python

# preprocessing for tags 
GameData['tags'] = GameData['tags'].str.strip('[]')
textfields = ['detailed_description', 'about_the_game', 'short_description', 'Top Reviews by Upvotes', 'tags']

def CleanText(DF, ColList, stop_words):
    '''
    Input: Dataframe containing text data, list of columns to clean, stopwords to remove
    Output: Dataframe containing cleaned text data
    '''
    CleanedCol = {}
    for col in ColList:
        stringlist = []
        for line in DF[col]:
            #remove html tags and text in brackets
            cleanstring = re.sub(re.compile('<.*?>') , '', str(line))
            cleanstring = re.sub(re.compile('\[.*?]') , ' ', cleanstring)
            cleanstring = re.sub(re.compile('\n') , ' ', cleanstring)
            #Remove stop words and sent to lowercase
            cleanstring = " ".join([word.lower() for word in str(cleanstring).split() if word not in stop_words])
            #Remove emojis and icons
            icons = re.compile("["u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002500-\U00002BEF"  # chinese char
                        "]+", re.UNICODE)
            cleanstring = re.sub(icons, '', cleanstring)
            #Remove punctuation
            for punctuation in string.punctuation: 
                cleanstring = cleanstring.replace(punctuation, '')
            
            stringlist.append(cleanstring)
        CleanedCol.update({col:stringlist})
    return pd.DataFrame(CleanedCol)

CleanData = CleanText(GameData, textfields, stop_words=stop_words)

In [29]:
CleanData


,detailed_description,about_the_game,short_description,Top Reviews by Upvotes,tags
0,young madam ai beast girl chinese girl insane ...,young madam ai beast girl chinese girl insane ...,get 500 diamonds start shopmaids different sty...,at glance adult content nudity f fxm sexual sc...,
1,during events halloween night sinister hallowe...,during events halloween night sinister hallowe...,as journalist receive secret email mountain pe...,so i sure i give positive negative review cons...,
2,during events halloween night sinister hallowe...,during events halloween night sinister hallowe...,as journalist receive secret email mountain pe...,so i sure i give positive negative review cons...,
3,also recommended you httpsstoresteampoweredco...,lots cuteness mess thats exactly get several k...,move pieces place untangle wool yarn cute kittens,a sweet little time waster want veg,cats 113 cute 107 minimalist 102 puzzle 98 poi...
4,furfly fascinating black white world traverse ...,furfly fascinating black white world traverse ...,help little furry friend escape evil swamp get...,this actually tough game play the controls sim...,2d 289 time attack 283 difficult 280 casual 27...
...,...,...,...,...,...
7304,dino eggs rebirth celebration expansion bestse...,dino eggs rebirth celebration expansion bestse...,dino eggs rebirth celebration expansion bestse...,im kind sucker new versions c64 games youth i ...,indie 33 strategy 31 action 23 casual 23 platf...
7305,you probably heard theory live inside computer...,you probably heard theory live inside computer...,physics sandbox game uses graphics card superc...,there literally nothing i would like give posi...,simulation 36 indie 33 early access 33 sandbox...
7306,quotgta chinatown wars dukes hazzard extimes...,quotgta chinatown wars dukes hazzard extimes...,free update state of emergency out now framed ...,what american fugitive when i started playing...,action 175 adventure 128 open world 125 indie ...
7307,for want enjoy quotmoe amp nudequot illustrati...,for want enjoy quotmoe amp nudequot illustrati...,for want enjoy quotmoe amp nudequot illustrati...,nan,nudity 21 indie 20 casual 20


In [26]:
Clean_GameData = pd.concat([GameData, CleanData], axis = 1)

In [27]:
from google.colab import files
Clean_GameData.to_csv('CleanSteamGameData.csv') 
files.download('CleanSteamGameData.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>